# Optimizing Financial Aid Resources

### Goals

This notebook uses the enrollment prediction model created elsewhere and optimizes the distribution of funds given the constraints involved (total scholarship funding, explainability).


### Process

* A. Define constraints, such as the average SAT score, or average high school GPA, the different scholarship values, and the number of students to admit.
* B. Load in student data, software libraries, and model objects.
* C. Pass 1 - Arbitrary Predictions
* D. Calculate and visualize yield, net revenue, and scholarship info per segment.
* E. Pass 2 - Transfer Learning
* F. Calculate and visualize yield, net revenue, and scholarship info in an explainable way.
* G. Save off the results for later use.


### Resources

* https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/pareto-optimized-road-trip/optimized-state-capitols-trip.ipynb

## Part A - Define Goals and Constraints

Next we're going to have to specify constraints we care about:

* **Maximum Student Enrollment** - The maximum number of domestic non-resident students you would like to enroll
* **Average Student SAT** - The minimum average SAT for domestic non-resident students you're willing to accept
* **Award Amounts** - The different scholarship amounts to award.
* **Scholarship Fund Amount** - The amount of money to distribute via Purple and Gold scholarships

#### First, Define Constraints

In [1]:
years_to_use = [2014, 2015, 2016]

In [2]:
fund_amount = 3000000  #3 million dollars
fund_amount = fund_amount * len(years_to_use)

maximum_student_enrollment = 700000

average_student_sat = 1090

tuition_amount = 34143

#scholarship_amounts = [0,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
scholarship_amounts = [0,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
max_scholarship = max(scholarship_amounts)

number_of_buckets = 20

## Part B - Load in student data, software libraries, and model objects

### Part B1 - Load software libraries

In [3]:
# Load in software libraries
import os
import sys
import pandas as pd
import numpy as np
import pickle
import random
import copy
from tqdm import tqdm

# Get data-munging libraries
from sklearn import preprocessing
from sklearn import decomposition

# Get ML libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier

# Get Viz libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Genetic algorithm
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

KeyboardInterrupt: 

### Part B2 - Load  objects

In [ ]:
model_location = '../propScoreMatching/forOpt/boosting_model.pkl'

with open(model_location, 'rb') as model_input:
    predict_model = pickle.load(model_input)

In [ ]:
scaler_location = '../propScoreMatching/forOpt/scaler.pkl'

with open(scaler_location, 'rb') as scaler_input:
    scaler = pickle.load(scaler_input)

In [ ]:
df = pd.read_csv('../propScoreMatching/forOpt/data_all.csv')

In [ ]:
df[['PurpleAndGoldAmount', 'GrantsAndScholarships', 'Tuition', 'yr']] = pd.read_csv('../propScoreMatching/forOpt/pugoTuitionAndGrants.csv')

In [ ]:
col_location = '../propScoreMatching/forOpt/columns.pkl'

with open(col_location, 'rb') as col_input:
    cols = pickle.load(col_input)

In [ ]:
cutoff_location = '../propScoreMatching/forOpt/best_cutoff.pkl'

cutoff_threshold = 0.5

useCutoff = True

if useCutoff:
    with open(cutoff_location, 'rb') as cutoff_input:
        cutoff_threshold = pickle.load(cutoff_input)

In [ ]:
df.columns

In [ ]:
cols

In [ ]:
predict_model.get_params()

### Part C - First Pass, Arbitrary Predictions

1. Get the data into a handy format
2. Add propensity scores
3. Predict students who will come with no PuGo
4. For each scholarship amount, predict students who will switch and attend

**Reminder Constraints**

fund_amount
average_student_sat
award_amounts
maximum_student_enrollment

*Coding Notes*

* df_2 is used for predictions (remove IsEnrolledOrMatriculated)
* df_3 has the results we use, the scholarship amounts

In [ ]:
y = df.IsEnrolledOrMatriculated
x = df.drop('IsEnrolledOrMatriculated', axis = 1)
#x = df.copy()

In [ ]:
x.tuition_after_grants.describe()

In [ ]:
x.tuition_after_grants.hist()

In [ ]:
rescale = True

if rescale:
    x_scaled = pd.DataFrame(scaler.transform(x[cols]), index = x.index, columns = cols)

In [ ]:
scalerValues = pd.Series(scaler.scale_)
scalerValues.index = cols
centerValues = pd.Series(scaler.center_)
centerValues.index = cols

In [ ]:
check = (df['TestScore'] == ((x_scaled['TestScore'] * scalerValues.TestScore) + centerValues.TestScore).round()).all()
check

In [ ]:
check = (x_scaled['TestScore'] == ((df['TestScore'] - centerValues.TestScore) / scalerValues.TestScore)).all()
check

## Part D - Calculate and visualize yield, net revenue, and scholarship info per segment.

### See how predictions fared

In [ ]:
enrollProb = predict_model.predict_proba(x_scaled)[:,1]
enrollPrediction = enrollProb >= cutoff_threshold

In [ ]:
actual_pugo_offered = df.PurpleAndGoldAmount.sum()
print 'PuGo Offered: ' + str(actual_pugo_offered)

actual_pugo_use = df[df.IsEnrolledOrMatriculated == 1].PurpleAndGoldAmount.sum()
print 'PuGo Use: ' + str(actual_pugo_use)

print 'PuGo use %: ' + str(float(actual_pugo_use) / actual_pugo_offered * 100)

In [ ]:
predicted_pugo_offered = df.PurpleAndGoldAmount.sum()
print 'Predicted PuGo Offered: ' + str(predicted_pugo_offered)

predicted_pugo_use = df[enrollPrediction == 1].PurpleAndGoldAmount.sum()
print 'Predicted PuGo Use: ' + str(predicted_pugo_use)

print 'Predicted PuGo use %: ' + str(float(predicted_pugo_use) / predicted_pugo_offered * 100)

In [ ]:
actual_student_count = df.IsEnrolledOrMatriculated.sum()
actual_total_student_count = len(df)

print 'Enrolled students: ' + str(actual_student_count)
print 'Admitted students: ' + str(actual_total_student_count)
print 'Actual yield: ' + str(actual_student_count * 100.0 / actual_total_student_count)

In [ ]:
predicted_student_count = enrollPrediction.sum()
total_student_count = len(enrollPrediction)

print 'Predicted students: ' + str(predicted_student_count)
print 'Admitted students: ' + str(total_student_count)
print 'Predicted yield: ' + str(predicted_student_count * 100.0 / total_student_count)

In [ ]:
predicted_net_revenue = df.tuition_after_grants[enrollPrediction == 1].sum()
print 'Predicted Net Revenue: ' + str(predicted_net_revenue)

In [ ]:
predicted_avg_sat = df.TestScore[enrollPrediction == 1].mean()
print 'Predicted Average SAT: ' + str(predicted_avg_sat)

## Part E - Pass 2, Transfer Learning and Genetic Optimization

* Useful resources, https://github.com/DEAP/notebooks/blob/master/OneMax.ipynb
* https://github.com/DEAP/notebooks/blob/master/SIGEvolution.ipynb
* http://deap.readthedocs.io/en/master/tutorials/basic/part1.html

### E1 - Assign students to 'buckets' based on academic performance

* Assign each student to a bucket
* Analyze and report on the characteristics of each bucket

In [ ]:
df['BucketRank'] = df['AcademicScore'].rank(ascending=1)

bucket_count = 20
df['Bucket'] = pd.cut(df['BucketRank'], bucket_count, labels = False)

In [ ]:
for i in range(0, bucket_count):
    high_school_gpa = df.HighSchoolGPA[df['Bucket'] == i].sum()
    min_high_school_gpa = df.HighSchoolGPA[df['Bucket'] == i].quantile(0.2)
    max_high_school_gpa = df.HighSchoolGPA[df['Bucket'] == i].quantile(0.8)
    test_score = df.TestScore[df['Bucket'] == i].sum()
    min_test_score = df.TestScore[df['Bucket'] == i].quantile(0.2)
    max_test_score = df.TestScore[df['Bucket'] == i].quantile(0.8)
    academic_score = df.AcademicScore[df['Bucket'] == i].sum()
    holistic_1 = df.HolisticScore1[df['Bucket'] == i].quantile(0.99)
    holistic_2 = df.HolisticScore2[df['Bucket'] == i].quantile(0.99)
    efc = df.EFC[df['Bucket'] == i].sum()
    students = df.HighSchoolGPA[df['Bucket'] == i].count()
    
    printString = ['bucket: ', str(i), '; students: ', str(students), '; hs GPA: ', str(high_school_gpa*1.0/students), 
     ' - ', str(min_high_school_gpa), '/', str(max_high_school_gpa), '; tests: ', str(test_score*1.0/students),
     ' - ', str(min_test_score), '/', str(max_test_score), '; scores: ', str(academic_score*1.0 / students),
     ' - ', str(efc*1.0/students), '/', str(holistic_1), '/', str(holistic_2)]
    print  ''.join(printString)

**Make a single function that evaluates a scholarship 'bucket list' and returns relevant metrics**

Note: this is the most important bit of code in this entire notebook.

The function returns:

   * net tuition revenue
   * scholarship amount
   * students enrolled
   * median test score
   * yield 

In [ ]:
def reconstruct(valuesIn, tag):
    #get unscaled values from scaled values
    return (valuesIn * scalerValues[tag]) + centerValues[tag]

In [ ]:
def deconstruct(valuesIn, tag):
    #get scaled values from unscaled values
    return (valuesIn - centerValues[tag]) / scalerValues[tag]

In [ ]:
def reCalculateTuition(dfIn):
    output = dfIn.copy()
    output['tuition_after_grants'] = output['Tuition'] - output['PurpleAndGoldAmount'] - output['GrantsAndScholarships']
    output.loc[output.tuition_after_grants < 0, 'tuition_after_grants'] = 0
    output['tuition_after_grants'] = deconstruct(output['tuition_after_grants'], 'tuition_after_grants')
    return output

In [ ]:
opt_x = x_scaled.copy()
opt_x[['PurpleAndGoldAmount', 'GrantsAndScholarships', 'Tuition']] = df[['PurpleAndGoldAmount', 'GrantsAndScholarships', 'Tuition']]

In [ ]:
test = reCalculateTuition(opt_x)['tuition_after_grants']
(test - opt_x.tuition_after_grants).describe()

In [ ]:
opt_x['PurpleAndGoldAmount'] = 10000 #testing for later
opt_x['Bucket'] = df['Bucket']

In [ ]:
opt_x['PredictionScore'] = predict_model.predict_proba(opt_x[cols])[:, 1]
opt_x['PredictionScoreTemp'] = opt_x['PredictionScore']
opt_x['EnrollPredictionTemp'] = predict_model.predict(opt_x[cols])

In [ ]:
opt_x.loc[opt_x.PredictionScoreTemp < cutoff_threshold, 'EnrollPredictionTemp'] = 0
opt_x.loc[opt_x.PredictionScoreTemp >= cutoff_threshold, 'EnrollPredictionTemp'] = 1
len(opt_x[opt_x['EnrollPredictionTemp'] == 1])

In [ ]:
opt_x['EnrollPredictionTemp'].describe()

In [ ]:
df['PurpleAndGoldAmount'].describe()

In [ ]:
def parse_scholarships_base(buckets, model, dfIn = opt_x, include_bucket_counts = False):
    """
        This function returns the net tuition revenue, scholarship used, 
        student count, and average test score
    """
    dfHold = dfIn.copy()
    tuition_revenue = 0
    scholarship_used = 0
    students = 0
    total_test_score = 0
    
    buckets = list(buckets)
    
    for i in range(len(buckets)):
        bucket_amount = buckets[i]
        dfHold.loc[dfHold.Bucket == i, 'PurpleAndGoldAmount'] = bucket_amount
    
    dfHold = reCalculateTuition(dfHold)   
    
    dfHold['PredictionScoreTemp'] = predict_model.predict_proba(dfHold[cols])[:, 1]
    dfHold.loc[dfHold.PredictionScoreTemp >= cutoff_threshold, 'EnrollPredictionTemp'] = 1
    dfHold.loc[dfHold.PredictionScoreTemp < cutoff_threshold, 'EnrollPredictionTemp'] = 0

    students = len(dfHold[dfHold['EnrollPredictionTemp'] == 1])
    #student_grads = df['5yr_grad_rate'][dfHold['EnrollPredictionTemp'] == 1].sum() #note ref to initial df
    scholarship_used = dfHold['PurpleAndGoldAmount'][dfHold['EnrollPredictionTemp'] == 1].sum()

    tuition_revenue =  (tuition_amount * dfHold['EnrollPredictionTemp']).sum() #df['5yr_grad_rate']
    test_scores = reconstruct(dfHold.TestScore, 'TestScore')
    avg_test_score = test_scores[dfHold['EnrollPredictionTemp'] == 1].mean()
    if np.isnan(avg_test_score):
        avg_test_score = 0
    total_students = dfHold.EnrollPredictionTemp.count()
    
    if include_bucket_counts:
        for i in range(len(buckets)):
            bucket_students = dfHold.EnrollPredictionTemp[(dfHold['EnrollPredictionTemp'] == 1) & (dfHold['Bucket'] == i)].count()
            bucket_scholarship_used = dfHold.PurpleAndGoldAmount[(dfHold['EnrollPredictionTemp'] == 1) & (dfHold['Bucket'] == i)].sum()
            bucket_revenue = bucket_students * tuition_amount
            bucket_test_score = test_scores[(dfHold['EnrollPredictionTemp'] == 1) & (dfHold['Bucket'] == i)].sum()
            bucket_total_students = dfHold.EnrollPredictionTemp[dfHold['Bucket'] == i].count()
            
            printString = ''.join(['Bucket: ', str(i), '; students: ', str(bucket_students), '; scholarships: ', 
                                  str(bucket_scholarship_used), '; gross: ', str(bucket_revenue - bucket_scholarship_used),
                                  '; test score: ', str(np.divide(bucket_test_score * 1.0, bucket_students * 1.0)), 
                                  '; student %age: ', str(np.divide(bucket_students * 1.0, bucket_total_students))])
            
            print printString
        print '\n\n'
        
    # return everything
    return tuition_revenue - scholarship_used, scholarship_used, students, avg_test_score, students * 1.0 / total_students

In [ ]:
def parse_scholarships(buckets, model, dfIn = opt_x):
    """
        This function returns the net tuition revenue, scholarship used, 
        student count, and average test score
    """
    net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships_base(buckets, model, dfIn, False)
    return net_revenue, scholarship_used, students, avg_test_score, student_yield

In [ ]:
buckets = np.zeros(bucket_count)
buckets = np.ones(bucket_count) * 1900
net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships(buckets, predict_model, opt_x)

print 'Net Revenue is ' + str(net_revenue)
print 'Scholarship Used is ' + str(scholarship_used)
print 'Student Count is ' + str(students)
print 'Average Test Score is ' + str(avg_test_score)
print 'Student Yield is ' + str(student_yield)

### Create the Evaluation Function

In [ ]:
def eval_scholarships(bucket_combo):
    """
        This function returns the net tuition revenue as well as the number of students
        predicted to enroll.
        
        The genetic algorithm will favor scholarship distributinon that
        bring more money or more students.
    """
    net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships(bucket_combo, predict_model)
    
    penalty = 100
    net_revenue_to_return = net_revenue
    if scholarship_used > fund_amount:
        net_revenue_to_return = 0#net_revenue_to_return * 1.0 * fund_amount / (scholarship_used * penalty)
        students = students / penalty
    elif students > maximum_student_enrollment:
        net_revenue_to_return = 0#net_revenue_to_return * 1.0 * maximum_student_enrollment / (students * penalty)
        students = students / penalty
    
    return net_revenue_to_return, avg_test_score
    #return float(students) * (float(avg_test_score) / 1600), students

### Create the mutation function

In [ ]:
np.random.choice([amt for amt in scholarship_amounts if amt > 10000],1)[0]

In [ ]:
def compare_lists(x, y):
    return 1 if len(frozenset(x).intersection(y)) == max(len(x),len(y)) else 0

In [ ]:
a = [1,2,3,5,4]
b = [1,2,3,4,5]

compare_lists(a,b)

In [ ]:
scholarship_amounts_2 = list(scholarship_amounts)
compare_lists(scholarship_amounts, scholarship_amounts_2)

In [ ]:
def mutate_scholarships_base(bucket_combo):
    original_combo = list(bucket_combo)

**Make a single function that takes a scholarship 'bucket list' and *changes* it to something very similar but not exactly the same**

Note: this is the second-most important bit of code in this entire notebook, after the evaluation function

In [ ]:
def mutation_operator(bucket_combo):
    """
        This function applies a random change to one set of scholarships
        
            - Init: create a bucket with 20 '0' values if it's not the right size
            - Increase: Add a scholarship to a random bucket
            - Reduce: Reduce a scholarship in a random bucket
            - Swap: Swaps the scholarship amounts of two buckets
    """
    possible_mutations = ['swap']
    original_combo = list(bucket_combo)
    bucket_list = list(bucket_combo)
    itercount = 0
    #print ('Bucketlist',bucket_list)
    #print ('Originalcombo',original_combo)
    #print ('Comparelists',compare_lists(original_combo, bucket_list))
    
    while compare_lists(original_combo, bucket_list) > 0: #keep going if it's not the same
        itercount = itercount + 1
        #print ('Itercount', itercount)
        if np.min(bucket_combo) < max_scholarship:
            possible_mutations.append('increase')
        if np.max(bucket_combo) > 0:
            possible_mutations.append('reduce')
        #
        mutation_type = random.sample(possible_mutations, 1)[0]
        #print 'Mutation type: ',mutation_type
        #
        # Increase mutation
        if mutation_type == 'increase':
            to_increase = True
            while to_increase:
                index_to_increase = random.randint(0, number_of_buckets - 1)
                if bucket_combo[index_to_increase] < (max_scholarship):
                    bucket_combo[index_to_increase] = np.random.choice([amt for amt in scholarship_amounts if amt > bucket_combo[index_to_increase]],1)[0]
                    to_increase = False
        #
        # Reduce mutation
        elif mutation_type == 'reduce':
            index_to_delete = random.randint(0, number_of_buckets - 1)
            if bucket_combo[index_to_delete] > 0:
                bucket_combo[index_to_delete] = np.random.choice([amt for amt in scholarship_amounts if amt < bucket_combo[index_to_delete]],1)[0]
        #
        # Swap mutation
        elif mutation_type == 'swap':
            index1 = random.randint(0, number_of_buckets - 1)
            index2 = index1
            while index2 == index1:
                index2 = random.randint(0, number_of_buckets - 1)
                if bucket_combo[index1] == bucket_combo[index2]:
                    index2 = index1
            #
            bucket_combo[index1], bucket_combo[index2] = bucket_combo[index2], bucket_combo[index1]
        #
        bucket_combo.sort()
        bucket_list = list(bucket_combo)
    # now that things are sorted, return the result
    return bucket_combo,

In [ ]:
test = np.arange(0, 20)
print test
print mutation_operator(test)

### Create the initial attribute function

In [ ]:
def random_scholarshipAmount():
    return np.random.choice(scholarship_amounts, 1)[0]

In [ ]:
def random_individual():
    random_buckets = np.random.choice(scholarship_amounts, size = number_of_buckets, replace = True).tolist()
    random_buckets.sort()
    return creator.Individual(random_buckets)

#### Type Creation

First step with DEAP is to create the required types. Usually the types created are the fitness and the individual. For the OneMax problem, we want to have a solution with as many ones as possible. Thus we need a maximizing fitness and a individual that is a sorted container, we'll choose the standard list.

Type creation is done by calling the function create in the creator module. This function takes two mandatory arguments and additional optional arguments. The first argument is the actual name of the type that we want to create. The second argument is the base classe that the new type created should inherit from. Finally, the optional arguments are members to add to the new type.

In [ ]:
creator.create('FitnessMulti', base.Fitness, weights=(0.99, 0.01))
creator.create('Individual', list, fitness=creator.FitnessMulti)

#creator.create('FitnessMax', base.Fitness, weights=(1.0))
#creator.create('Individual', list, fitness=creator.FitnessMax)

In [ ]:
all_buckets = range(0, bucket_count) #there are 20 buckets to start with
bucket_combinations = {}

buckets = [0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
ind = creator.Individual([1, 0, 1, 1, 0])

print(ind)
print(type(ind))
print(type(ind.fitness))
print(type(all_buckets))

#### Toolbox

The toolbox is intended to store functions with their arguments under standard aliases for uses in algorithms. Functions are registered by a function call with two mandatory arguments, the alias to give to the function and the function it will be associate with. Any additional argument will be given as argument when the alias is called.

In [ ]:
toolbox = base.Toolbox()

This first one, attr_bool, calls randint from the random module with arguments (0, 1) to create an integer in the interval $[0, 1]$.

In [ ]:
toolbox.register("attr_scholarshipAmount", random_scholarshipAmount)

In [ ]:
bit = toolbox.attr_scholarshipAmount()
print("bit is of type %s and has value\n%s" % (type(bit), bit))

The second function, individual, when called, will use the initRepeat function made available in the tools module to fill an Individual class with what is produced by 10 calls to the previously defined attr_scholarship function. 

* Note: this has been deprecated in favor of a custom function, random_individual, which returns sorted results

In [ ]:
toolbox.register("individual", random_individual)

In [ ]:
ind = toolbox.individual()
print("ind is of type %s and contains %d bits\n%s" % (type(ind), len(ind), ind))

The same thing is done for the population function.

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
pop = toolbox.population(n = 3)
print("pop is of type %s and contains %d individuals\n%s" % (type(pop), len(pop), pop))

## Genetic Operators

Registering the operators and their default arguments in the toolbox is done as follows:

The evaluation is given the alias evaluate. Having a single argument being the individual to evaluate we don’t need to fix any, the individual will be given later in the algorithm. 

In [ ]:
toolbox.register('evaluate', eval_scholarships)

In [ ]:
ind = toolbox.individual()
evaluation = toolbox.evaluate(ind)
print("Eval is of type %s and contains \n%s" % (type(evaluation), evaluation))

The two points crossover function is registered the same way under the alias mate. 

In [ ]:
toolbox.register("mate", tools.cxTwoPoint)

The mutation, for its part, needs an argument to be fixed (the independent probability of each attribute to be mutated indpb). In the algorithms the mutate() function is called with the signature toolbox.mutate(mutant). This is the most convenient way because each mutation takes a different number of arguments, having those arguments fixed in the toolbox leave open most of the possibilities to change the mutation (crossover, selection, or evaluation) operator later in your researches.

In [ ]:
toolbox.register('mutate', mutation_operator)

We can for example mutate an individual and expect 10% of its attributes to be flipped.

In [ ]:
ind = toolbox.individual()
print(ind)
for i in range(0,4):
    ind = toolbox.mutate(ind)[0]
    print ind
    revenue = toolbox.evaluate(ind)
    #print 'Revenue of ' + str(revenue) + 'comes from ' + str(ind)

Finally, the selection operator is registered under the name select and the size of the tournament set to 3.

* https://pdfs.semanticscholar.org/2fb8/850864d71d6b9b30a6f4729a82cba456b027.pdf
* http://deap.readthedocs.io/en/master/api/tools.html

In [ ]:
toolbox.register("select", tools.selTournament, tournsize = 5)

### Evolving the Population

The main program shall be defined in a main function. All previous declarations are put in the global scope of the module to make created objects available for further use in other user experiments.

The main program is very simple. It consists of generating a population, and giving it to the algorithm for it to evolve a solution. Here we will employ some helpful introspection tools such as Statistics and a Hall of Fame. The statistics are computed using numpy functions on the population, and the hall of fame keeps track of the best individuals that ever appeared during the evolution. The algorithm take as arguments, among other, the population and the toolbox. Once the evolution is finished the population contains the individuals from the last generation.

In [ ]:
pop = toolbox.population(n = 50)

In [ ]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)
stats.register("min", np.min)

* http://deap.readthedocs.io/en/master/api/algo.html

In [ ]:
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb = 0.25, mutpb = 0.8, ngen = 30, stats = stats, halloffame = hof, verbose = True)

#cxpb 0.5 or 0? - The probability of mating two individuals.
#mutpb 0.2 or 1.0? - The probability of mutating an individual.

In [ ]:
print("Best individual is: %s with fitness: %s" % (hof[0], hof[0].fitness))

#print ('Evaluation of best individual:', parse_scholarships(hof[0]))
print ('\nBucket, Students, ScholarshipUsed, NetRevenue, AvgTestScore, Yield')
print ('Evaluation of best individual detailed:', parse_scholarships_base((hof[0]), predict_model, include_bucket_counts = True))

import matplotlib.pyplot as plt
gen, avg, max_ = logbook.select("gen", "avg", "max")
plt.plot(gen, avg, label="average")
plt.plot(gen, max_, label="maximum")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="lower right")
plt.show()

# Check results

In [ ]:
best_buckets = hof[0]

In [ ]:
bucketDF = pd.DataFrame({'newPugoAmount' : best_buckets, 'Bucket' : range(0, 20)})
bucketDF

In [ ]:
evalDF = x_scaled.copy()
evalDF[['PurpleAndGoldAmount', 'GrantsAndScholarships', 'Tuition', 'Bucket']] = df[['PurpleAndGoldAmount', 'GrantsAndScholarships', 'Tuition', 'Bucket']]

In [ ]:
evalDF = evalDF.merge(bucketDF, on = 'Bucket', how = 'left')

In [ ]:
evalDF.Bucket.value_counts()

In [ ]:
evalDF['IsEnrolledOrMatriculated'] = df.IsEnrolledOrMatriculated
evalDF['PredictionScore'] = predict_model.predict_proba(evalDF[cols])[:, 1]
evalDF['OldPredictions'] = evalDF['PredictionScore'] >= cutoff_threshold
evalDF['TuitionAfterGrantsOld'] = evalDF.tuition_after_grants
evalDF['tuition_after_grants'] = evalDF['Tuition'] - evalDF['newPugoAmount'] - evalDF['GrantsAndScholarships']
evalDF['NewPredictionScore'] = predict_model.predict_proba(evalDF[cols])[:, 1]
evalDF['NewPredictions'] = evalDF['NewPredictionScore'] >= cutoff_threshold

In [ ]:
cols

In [ ]:
evalDF.head()

In [ ]:
(evalDF.NewPredictionScore == evalDF.PredictionScore).all()

In [ ]:
(evalDF.TuitionAfterGrantsOld == evalDF.tuition_after_grants).all()

In [ ]:
(evalDF.PurpleAndGoldAmount == evalDF.newPugoAmount).all()

In [ ]:
evalDF[evalDF.IsEnrolledOrMatriculated == 1].newPugoAmount.sum()

In [ ]:
evalDF['OldPredictions'].sum()

In [ ]:
evalDF['NewPredictions'].sum()

In [ ]:
c = evalDF[evalDF.NewPredictionScore == evalDF.PredictionScore]
len(c)

In [ ]:
len(evalDF)

In [ ]:
k = evalDF[evalDF.NewPredictions == False]
reconstruct(k.TestScore, 'TestScore').mean()

In [ ]:
k = evalDF[evalDF.NewPredictions]
reconstruct(k.TestScore, 'TestScore').mean()

In [ ]:
evalDF.newPugoAmount.sum()